<a href="https://colab.research.google.com/github/shitlime/build-trime/blob/main/Build_Trime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境测试

In [ ]:
!echo "build trime"
!uname -a
!git --version
!echo "=================apt upgrade====================="
!apt upgrade

# 编译Trime

*完整编译一次Trime需要30min以上*

## 1. 获取源代码

In [ ]:
#@markdown # **Git Clone**

#@markdown 在此处输入源代码仓库地址（一般使用github仓库）
import os
import urllib.request

REPO_URL = "https://github.com/osfans/trime"  #@param {type: "string"}

# 检查仓库地址是否有效
try:
  urllib.request.urlopen(REPO_URL)
except urllib.error.URLError:
  print("仓库地址无效！")

# 如果./trime存在，先删除
if os.path.exists("./trime"):
  !rm -rf ./trime

# 开始克隆
!git clone {REPO_URL} trime --recurse-submodules
!echo ">>>>>> 克隆完成"

## 2. 安装依赖

In [ ]:
#@markdown #1. 安装依赖
!apt install openjdk-21-jdk
!apt-get install android-sdk
!apt-get install ninja-build
!bash ./script/dependency.sh  # trime的依赖安装脚本（似乎是空的）
!apt install -y clang-format


#@markdown #2. 安装并配置Android命令行工具

# 路径变量
ANDROID_SDK_DIR = !whereis android-sdk
ANDROID_SDK_DIR = ANDROID_SDK_DIR[0].split(": ")[1]

#  下载并解压
# @markdown 输入android命令行工具的下载地址（ **如果可以使用就不用修改** ）：
ANDROID_CMD_TOOLS_URL = "https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip"  #@param {type: "string"}
!curl {ANDROID_CMD_TOOLS_URL} -o commandlinetools.zip
!unzip commandlinetools.zip

#  移动到sdk安装目录
!mkdir -p {ANDROID_SDK_DIR}/cmdline-tools/latest
!mv cmdline-tools/* {ANDROID_SDK_DIR}/cmdline-tools/latest

#  接受协议
!yes | {ANDROID_SDK_DIR}/cmdline-tools/latest/bin/sdkmanager --licenses

## 3. 编译

In [ ]:
# @markdown # **重要： 编译前的一些配置**

# 移动到trime根目录
%cd trime

# 添加git用户名，缺失会报错
# @markdown git用户名（必须，否则报错）：
GIT_USER_NAME = "user_name"  #@param {type: "string"}
!git config --global user.name {GIT_USER_NAME}

# 编写local.properties文件，指定SDK路径
!touch local.properties
!echo "sdk.dir={ANDROID_SDK_DIR}" > local.properties
!cat local.properties

### 3.1 （可选）编译 Debug 版

In [ ]:
!make debug

### 3.2 编译 Release 版

In [ ]:
# @markdown # Release版需要签名

# 生成release签名密钥
# @markdown **输入签名密钥信息：**
FIRST_AND_LAST_NAME="Tiansuo Li" #@param {type: "string"}
ORGANIZATION_UNIT="Personal" #@param {type: "string"}
ORGANIZATION="Tiansuo Li" #@param {type: "string"}
LOCALITY="Shimokitazawa" #@param {type: "string"}
PROVINCE="Tokyo" #@param {type: "string"}
COUNTRY_CODE="JP" #@param {type: "string"}
SIGNING_KEY_ALIAS="google-colab-build" #@param {type: "string"}
SIGNING_STORE_PASSWORD="114514" #@param {type: "string"}
SIGNING_KEY_PASSWORD="114514" #@param {type: "string"}

# 生成签名密钥
%cd app  # 在2024-01-17的trime中，密钥文件目录在 `trime/app` 下
if not os.path.exists("my-release-key.keystore"):
  !touch tempfile
  !echo {SIGNING_STORE_PASSWORD} >> tempfile
  !echo {SIGNING_STORE_PASSWORD} >> tempfile
  !echo {FIRST_AND_LAST_NAME} >> tempfile
  !echo {ORGANIZATION_UNIT} >> tempfile
  !echo {ORGANIZATION} >> tempfile
  !echo {LOCALITY} >> tempfile
  !echo {PROVINCE} >> tempfile
  !echo {COUNTRY_CODE} >> tempfile
  !echo "y" >> tempfile
  !cat tempfile | keytool -genkey -v -keystore my-release-key.keystore -alias {SIGNING_KEY_ALIAS} -keyalg RSA -keysize 2048 -validity 10000
  !rm -f tempfile
%cd ..


# 把密钥信息写入配置文件
if os.path.exists("keystore.properties"):
  !rm -f keystore.properties
!touch keystore.properties
!echo "storeFile=my-release-key.keystore" >> keystore.properties
!echo "keyAlias={SIGNING_KEY_ALIAS}" >> keystore.properties
!echo "storePassword={SIGNING_STORE_PASSWORD}" >> keystore.properties
!echo "keyPassword={SIGNING_KEY_PASSWORD}" >> keystore.properties

# 开始编译
!make release

### 3.3 （可选）清理编译

+ 需要获取结果时不推荐清理
+ 清理的主要用途是编译失败 或 编译debug和release版时避免影响

In [ ]:
!make clean

## 4. 获取结果

运行打包结果的代码（下方的格子）后

*如果不需要打包完整项目，可以使用 `#` 注释打包完整项目的代码，节省较多的时间*


### 4.1 需要获取编译后的完整项目用于开发（完整的trime项目文件夹）

到左侧 `文件` -> `trime.tgz` -> `鼠标右键` -> `下载`


### 4.2 需要获取编译后的APK文件（安卓安装包文件）

到左侧 `文件` -> `Trime-APK.tgz` -> `鼠标右键` -> `下载`


### 4.3 需要获取编译后的prebuilt文件用于开发（开发时使用的jni部分预编译文件）

到左侧 `文件` -> `prebuilt.tar` -> `鼠标右键` -> `下载`

In [ ]:
# @markdown 打包结果
%cd ..
# 打包完整项目（注释掉可以提速）
!tar -zcvf trime.tar

# 打包APK安装包
!tar -zcvf Trime-APK.tar trime/app/build/outputs/apk

# 打包prebuilt相关文件（之后需要自行处理）
!tar -zcvf prebuilt.tar trime/app/build/intermediates/stripped_native_libs